In [ ]:
import pandas as pd

In [ ]:
df=pd.read_csv("input.csv")

In [ ]:
#define extract name country function

def extract_country(io):
    arr=io.split(' ')
    return arr[1]

In [ ]:
#define create supply vendor lookup table from lookup file

ldf=pd.read_csv("supply-vendor-lookup.csv")
ldf=ldf.set_index("Name",drop=True)

In [ ]:
#create dataframes for each country

df['Country']=df.apply(lambda row:extract_country(str(row['Campaign'])),axis=1)
dftw=df[df['Country']=='Taiwan']
dfmy=df[df['Country']=='Malaysia']
dfsg=df[df['Country']=='Singapore']

In [ ]:
#define calculate CPA function

def getCPA(df):
    total_cost=df['Partner Cost (USD)'].sum()
    total_conv=df['01 - Total Click + View Conversions'].sum()
    return total_cost/total_conv

In [ ]:
#define apply spend threshold to each country df

def apply_spend_threshold(df,agg_cpa):
    df=df[df['Partner Cost (USD)']>agg_cpa]
    return df

In [ ]:
#def clean data function

def clean_data(df):
    df=df[df['Partner Cost (USD)']!=0]
    df=df[df['Browser']!='']
    df=df[df['Device Type']!='']
    df=df[df['Operating System']!='']
    df=df[df['Supply Vendor']!='']
    df = df.dropna()
    return df

In [ ]:
def add_cpa_column(df):
    df['CPA']=df.apply(lambda row: row['Partner Cost (USD)']/row['01 - Total Click + View Conversions'] if row['01 - Total Click + View Conversions']!=0 else 0,axis=1)
    return df

In [ ]:
def add_bid_factor(df,global_cpa):
    df['BidAdjustment']=df.apply(lambda row: row['CPA']/global_cpa,axis=1)
    return df

In [ ]:
def drop_columns(df):
    df=df.drop(['Campaign','Partner Cost (USD)','01 - Total Click + View Conversions','Country','CPA'],axis=1)
    return df

In [ ]:
def rename_columns(df):
    df=df.rename(columns={"Device Type": "DeviceType","Operating System":"Os","Supply Vendor":"SupplyVendorId","User Hour Of Week":"HourOfWeek"})
    return df

In [ ]:
def add_supply_vendor_id_column(df,lookup):
    df["Supply Vendor"]=df.apply(lambda row:str(str(str(row["Supply Vendor"]).lower()).replace(" ","")),axis=1)
    df['Supply Vendor']=df.apply(lambda row: lookup.loc[str(row['Supply Vendor']),"ID"],axis=1)
    return df

In [ ]:
def create_bid_list(df,name,lookup):
    agg_cpa=getCPA(df)
    df=clean_data(df)
    df=apply_spend_threshold(df,agg_cpa)
    df=add_cpa_column(df)
    df=add_bid_factor(df,agg_cpa)
    df=add_supply_vendor_id_column(df,lookup)
    df=drop_columns(df)
    df=rename_columns(df)
    df.to_csv(name+'.csv',index=False)
    return df

In [ ]:
create_bid_list(dftw,'tw-bid-list',ldf)
create_bid_list(dfmy,'my-bid-list',ldf)
create_bid_list(dfsg,'sg-bid-list',ldf)